In [12]:
# importing the necessary libararies for the program

import numpy as np
import sympy as sp
from tabulate import tabulate

In [ ]:
def compute_jacobian(f_exprs, variables):
    """
    Computes the symbolic jacobian matrix for a system of equations

    Parameters:
    f_exprs: List of functions in the sympy symbolic format
    variables: List of varibales in the sympy symbolic format

    Returns:
    A sympy matrix representingthe Jacobian
    """
    # Create the jacobian by differentiating each function with respect to each variable
    jacobian_matrix = sp.Matrix([[sp.diff(f,var) for var in variables] for f in f_exprs])
    return jacobian_matrix

def system_function_numeric(f_exprs, variables):
    """
    Converts a list of symbloic expressions into numerical functions

    Parameters:
    f_exprs: List of functions in the sympy symbolic format
    variables: List of varibales in the sympy symbolic format

    Returns:
    A system that takes numerical input for the variables and returns the evaluated system
    """
    # sp.lambdify converts the symbolic representation to a function and returns a tuple of values
    F_func = sp.lambdify(variables,f_exprs,'numpy')
    return F_func

def jacobian_numeric(f_exprs, variables):
    """
    Converts the symbolic jacobian to a numerical function
    Parameters:
    f_exprs: List of functions in the sympy symbolic format
    variables: List of varibales in the sympy symbolic format

    Returns:
    A function, that given the numerical values returns the Jacobian as a numpy array
    """
    J_sym = compute_jacobian(f_exprs,variables)
    J_func = sp.lambdify(variables,J_sym,'numpy')
    return J_func

def newton_raphson(f_exprs, variables, initial_guess, tol = 1e-9, max_iter=5):
    """
    Solves the system of equations F(X) = 0 using the Newton-Raphson method

    Parameters:
    f_exprs: List of functions in the sympy symbolic format
    variables: List of varibales in the sympy symbolic format
    initial_guess: List of initial guess values
    tol: Tolerance of convergence
    max_iter: Maximum number of iterations

    Returns:
    The approximated solution as a NumPy array
    """
    # Convert the symbolic system and the Jacobian to numerical functions
    F_func = system_function_numeric(f_exprs, variables)
    J_func = jacobian_numeric(f_exprs, variables)

    # initializing the initial_guess array x(0)
    X = np.array(initial_guess, dtype=float)
    
    # a table to store data for storing the output
    table_data = []
    
    # initializing the output file
    f = open("Shivansh_Yadav_230977_A4_output.txt","w+")

    # initializing the norm variable
    euclid_norm = "-------"

    # iterations to solve the system
    for i in range(max_iter):

        table_data.append([i,X[0],X[1],X[2],euclid_norm])
        # evaluating the numerical values of the funciton system and the jacobian
        F_val = np.array(F_func(*X),dtype=float)
        J_val = np.array(J_func(*X), dtype=float)

        
        """
        Here we use the algorithm of newton raphson method:
        J*delta + F = 0
        J: the jacobian at some X (system of variables)
        F: system of expressions at some X 
        delta: value used to calculate the next variable value matrix for the iteration

        we use:
        delta = (J^-1)*(-F)

        to find the value of delta
        """
        delta = np.linalg.solve(J_val,-F_val)

        # initializing the new variable system
        X_new = X + delta

        # calculating the euclidian norm
        euclid_norm = np.linalg.norm(delta, ord=2)

        # if norm is smaller than the tolerance
        if euclid_norm < tol:
            # entering the data in the table and the output file
            if(euclid_norm<tol):
                euclid_norm = 0
            table_data.append([i+1,X_new[0],X_new[1],X_new[2], euclid_norm])
            table = tabulate(table_data, headers = ['k', 'x1(k)', 'x2(k)', 'x3(k)', '||x(k) - x(k-1)||'], tablefmt="grid")
            print(table)
            f.write(table)
            f.close()
            print(f"Converged at {i+1} iterations")
            return X_new
        # updating the variable system
        X = X_new
    


    # this is the case where the data doesn't converge but we have limited iteration values so this won't be needed
    # entering the data in the table and the output file
    table_data.append([i+1,X_new[0],X_new[1],X_new[2], euclid_norm])
    table = tabulate(table_data, headers = ['k', 'x1(k)', 'x2(k)', 'x3(k)', '||x(k) - x(k-1)||'],tablefmt="grid")
    print(table)
    f.write(table)
    f.close()
    print("Did not converge within the maximum number of iterations")
    return X

In [14]:
# The main loop

# initializing symbols
x1, x2, x3 = sp.symbols('x1 x2 x3')

# initializing functions
f1 = 3*x1 - sp.cos(x2*x3) -0.5
f2 = x1**2 - 81*((x2+0.1)**2) + sp.sin(x3) + 1.06
f3 = sp.exp(-x1*x2) + 20*x3 + (10*sp.pi -3)/3

# initializing the array of symbolic functions
f_exprs = [f1,f2,f3]

# initializing the array of symbolic variables
variables = [x1,x2,x3]

# initializing the array of initial_guess
initial_guess = [0.1, 0.1, -0.1]

# Calculating the solution
solution = newton_raphson(f_exprs,variables,initial_guess)
print("Solution:",solution)

+-----+----------+-------------+-----------+------------------------+
|   k |    x1(k) |       x2(k) |     x3(k) | ||x(k) - x(k-1)||      |
+=====+==========+=============+===========+========================+
|   0 | 0.1      | 0.1         | -0.1      | -------                |
+-----+----------+-------------+-----------+------------------------+
|   1 | 0.49987  | 0.0194668   | -0.52152  | 0.5865670056112852     |
+-----+----------+-------------+-----------+------------------------+
|   2 | 0.500014 | 0.00158859  | -0.523557 | 0.017994451377116805   |
+-----+----------+-------------+-----------+------------------------+
|   3 | 0.5      | 1.24448e-05 | -0.523598 | 0.001576755749180865   |
+-----+----------+-------------+-----------+------------------------+
|   4 | 0.5      | 7.75786e-10 | -0.523599 | 1.2448781083989356e-05 |
+-----+----------+-------------+-----------+------------------------+
|   5 | 0.5      | 2.20432e-18 | -0.523599 | 7.760833040843749e-10  |
+-----+----------+--